In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [2]:
loaded_model = tf.keras.models.load_model("recommender_model_savedmodel")

In [4]:
# Load data
rating = pd.read_csv("archive (1)/tourism_rating.csv")
place = pd.read_csv("archive (1)/Dataset_NusaGo - en-ver (1).csv")
user = pd.read_csv("archive (1)/user.csv")

# Merge dataframes

merged_df = pd.merge(rating, place, how="inner", on="Place_Id")

merged_df = pd.merge(merged_df, user, how="inner", on="User_Id")

merged_df = merged_df.drop(["picture", "category", "Location", "Age"], axis=1)


# Convert "User_Id" and "Place_Id" to strings

merged_df["User_Id"] = merged_df["User_Id"].astype(str)

merged_df["Place_Id"] = merged_df["Place_Id"].astype(str)

merged_df["place_description"] = merged_df["place_description"].astype(str)

merged_df["label"] = merged_df["label"].astype(str)

In [8]:
def get_embedding(model, string):
    return model.layers[1](model.layers[0](string))


# Function to get recommendations based on multiple keywords and category
def get_recommendations(model, user_id, keywords, category=None, top_k=5):
    user_model = model.user_model
    place_model = model.place_model
    description_model = model.description_model

    # Get embeddings for each keyword
    keyword_embeddings = [
        get_embedding(description_model, np.array([kw])) for kw in keywords
    ]

    # Calculate average embedding for keywords
    avg_keyword_embedding = tf.reduce_mean(keyword_embeddings, axis=0)

    # Get all place embeddings
    all_place_embeddings = get_embedding(place_model, merged_df["Place_name"].unique())

    # Filter places based on category (if provided)
    if category is not None:
        filtered_places = merged_df[merged_df["label"] == category][
            "Place_name"
        ].unique()
        all_place_embeddings = get_embedding(place_model, filtered_places)

    # Calculate cosine similarity between average keyword embedding and filtered place embeddings
    similarities = tf.matmul(all_place_embeddings, tf.transpose(avg_keyword_embedding))

    # Get indices of top-k places with highest similarity
    top_indices = tf.argsort(similarities[:, 0], direction="DESCENDING")[:top_k]

    # Get recommended places and corresponding user_ids
    recommended_places = (
        filtered_places[top_indices]
        if category is not None
        else merged_df["Place_name"].unique()[top_indices]
    )
    user_ids = [user_id] * top_k

    return recommended_places, user_ids


# Example usage with multiple keywords and category filter
user_id = "78"  # Replace with the desired user_id
keywords = [
    "White Sand Beach",
    "Waterfall",
    "Family water tourism",
]  # Replace with the desired keywords
category = "water activities"  # Replace with the desired category

recommendations, user_ids = get_recommendations(
    loaded_model, user_id, keywords, category
)

# Print recommendations
print(
    f"Rekomendasi Tempat untuk Pengguna dengan ID {user_id}, Kata Kunci '{keywords}', dan Kategori {category}:"
)
for i, place_name in enumerate(recommendations, start=1):
    print(f"{i}. {place_name}")

Rekomendasi Tempat untuk Pengguna dengan ID 78, Kata Kunci '['White Sand Beach', 'Waterfall', 'Family water tourism']', dan Kategori water activities:
1. Progo rafting
2. Sri Gethuk Waterfalls
3. Sembukan Beach
4. Semirang Waterfalls
5. Goa Cemara Beach
